In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from threading import Thread
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
        #print(os.path.join(dirname, filename))
#        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_dir = "/kaggle/input/predict-volcanic-eruptions-ingv-oe/train"
train_filenames = os.listdir(train_dir)


In [ ]:
train_filename = train_filenames[0]
data = pd.read_csv( os.path.join(train_dir,train_filename), dtype="Int16")
data.describe()

In [ ]:
data.memory_usage().sum()/(10**6)

# Checando que tan buena estuvo la prank

Según distintos archivos, hay ciertos que no tienen valores registrados (NaNs) o cosas raras como puros ceros. Sería importante ver como está dicha situación para todos los datos

In [ ]:
def check_prank(data):
    """Function to check if some columns contain only nans or some constant"""
    nrows, _ = data.shape
    only_nans = [data[sensor].isna().sum() for sensor in data.columns]
    only_constants = [data[sensor].std() == 0 for sensor in data.columns]
    
    return only_nans, only_constants

In [ ]:
def yield_partition(data, npartitions):
    n_elements = len(data)
    n = n_elements//npartitions
    
    for k in range(npartitions):
        i = k*n
        j = (k+1)*n
        if k != npartitions - 1:
            yield data[i:j]
        else:
            yield data[i:]  # Last one

In [ ]:
def parallelize(nthreads):
    def decorator(original_function):
        def parallelized_function(dir_path, filenames, verbose=True):
            parallel_ans = []
            threads = []
            counter = 1
    
            # Partir los datos
            for partition in yield_partition(filenames, nthreads):
                thread = Thread(target=original_function, args=(dir_path, partition, verbose, parallel_ans))
                threads.append(thread)
                print("Starting",counter)
                thread.start()
                counter += 1 
            
            for thread in threads:
                thread.join()
            
            list_nans = [df_nan for df_nan, df_constant in parallel_ans]
            list_constants = [df_constant for df_nan, df_constant in parallel_ans]
            concatenation_nans = pd.concat(list_nans)
            concatenation_constants = pd.concat(list_constants)
            return concatenation_nans, concatenation_constants
        
        return parallelized_function
    
    return decorator

In [ ]:
@parallelize(4)
def check_pranks(dir_path, filenames, verbose=True, parallel_ans = None):
    nans = []
    constants = []
    for i,filename in enumerate(filenames,1):
        df = pd.read_csv(os.path.join(dir_path,filename), dtype="Int16")
        only_nans, only_constants = check_prank(df)
        nans.append(only_nans)
        constants.append(only_constants)
        if verbose and i%10 == 0:
            print(i)
    
    df_nans = pd.DataFrame(nans, columns=df.columns)
    df_constants = pd.DataFrame(constants, columns=df.columns)
    
    if parallel_ans != None:
        parallel_ans.append( (df_nans, df_constants) )
    else:
        return df_nans, df_constants

## Datos de entrenamiento 

In [ ]:
%%time
df_train_nans, df_train_constants = check_pranks(train_dir, train_filenames)

In [ ]:
df_train_nans.to_csv("df_train_nans.csv", index=False)

In [ ]:
df_train_nans.sum(axis=0)

In [ ]:
df_train_constants.sum(axis=0)

In [ ]:
df_train_constants.to_csv("df_train_constants.csv", index=False)

## Datos de prueba

In [ ]:
test_dir = "/kaggle/input/predict-volcanic-eruptions-ingv-oe/test"
test_filenames = os.listdir(test_dir)

In [ ]:
%%time
df_test_nans, df_test_constants = check_pranks(test_dir, test_filenames)

In [ ]:
df_test_nans.to_csv("df_test_nans.csv", index=False)
df_test_constants.to_csv("df_test_constants.csv", index=False)

In [ ]:
df_test_nans.sum(axis=0)

In [ ]:
df_test_constants.sum(axis=0)

----------------
# Zona después del \end{document}

`time_to_eruption` es el número de muestras antes de la siguiente erupción